In [1]:
!pip install transformers accelerate bert-score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.8 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 80.5 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.9.41
    Uninstalling nvidia-nvjitlink-cu12-12.9.41:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.9.41
  Attempting uninstall: nvidia-curand-cu12
    Found existing in

In [ ]:
!pip uninstall -y pylibcudagraph-cu12 rmm-cu12
!pip install transformers  # Use latest version for bakLlava compatibility
!pip install scikit-learn
!pip install timeout-decorator  # Install timeout-decorator

import pandas as pd
from transformers import AutoProcessor, LlavaForConditionalGeneration
from PIL import Image
import torch
from tqdm import tqdm
import gc
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
import os
from sklearn.metrics import accuracy_score, f1_score
from timeout_decorator import timeout, TimeoutError  # Import timeout decorator

# Load the CSV file
csv_path = "/kaggle/input/dataset-curated-with-split-r/Sf/Sf_qa_data_trimmed_test_r.csv"
pred_path = "/kaggle/working/bakllava_vqa_predictions_new.csv"
metrics_path = "/kaggle/working/bakllava_vqa_metrics_new.csv"
df = pd.read_csv(csv_path, header=None, names=["image_path", "question", "ground_truth"])

# Load bakLlava processor and model
model_id = "llava-hf/bakLlava-v1-hf"
processor = AutoProcessor.from_pretrained(model_id)
model = LlavaForConditionalGeneration.from_pretrained(model_id, torch_dtype=torch.float16)

# Move model to GPU
device = torch.device("cuda")
model.to(device)
model.eval()

# List to store skipped entries
skipped_entries = []

# Function to predict answer with timeout
@timeout(10)  # Set timeout to 10 seconds
def predict_answer(image_path, question):
    try:
        if not os.path.exists(image_path):
            print(f"Image not found: {image_path}")
            return ""
        image = Image.open(image_path).convert("RGB")

        # Prepare the VQA prompt for bakLlava
        prompt = f"[INST] <image> Question: {question} Answer in one word: [/INST]"
        inputs = processor(text=prompt, images=image, return_tensors="pt").to(device, torch.float16)

        with torch.no_grad():
            outputs = model.generate(**inputs, max_new_tokens=20)
        predicted_answer = processor.decode(outputs[0], skip_special_tokens=True).strip()

        # Extract one-word answer (post-process if needed)
        predicted_answer = predicted_answer.split()[-1]  # Take the last word as the answer

        del inputs, outputs
        gc.collect()
        torch.cuda.empty_cache()
        return predicted_answer
    except Exception as e:
        print(f"Error processing {image_path}: {e}")
        return ""

# Resume support
start_idx = 0
if os.path.exists(pred_path):
    existing = pd.read_csv(pred_path)
    start_idx = len(existing)
    print(f"Resuming from index {start_idx}")
else:
    existing = pd.DataFrame()

# Lists to store predictions and ground truth
predictions = []
ground_truths = df["ground_truth"].tolist()
y_true = []
y_pred = []
results = []

# Predict answers for each row
for idx, row in tqdm(df.iterrows(), total=len(df), desc="Processing"):
    if idx < start_idx:
        continue  # Ignore those already done
    # if idx == 100:
    #     break  # Ignore those already done
    full_image_path = f"/kaggle/input/dataset-curated-with-split-r/{row['image_path']}"
    question = row["question"]
    try:
        predicted = predict_answer(full_image_path, question)
    except TimeoutError:
        print(f"Timeout processing row {idx}: {full_image_path}")
        skipped_entries.append({"row": idx, "image_path": full_image_path, "question": question})
        predicted = ""  # Skip this entry
    predictions.append(predicted)

    y_true.append(str(ground_truths[idx]).lower())
    y_pred.append(str(predicted).lower())

    results.append({
        "img_path": full_image_path,
        "question": question,
        "true_answer": str(ground_truths[idx]).lower(),
        "predicted_answer": str(predicted).lower()
    })
    if idx <= 100:
        match = "✅" if y_pred[-1] == y_true[-1] else "❌"
        print(f"[{idx}] Truth: {y_true[-1]} | Predicted: {y_pred[-1]} | Match: {match}")

    # Save every 1000 or last item
    if (idx + 1) % 1000 == 0 or (idx + 1) == len(df):
        # Convert to DataFrame
        chunk_df = pd.DataFrame(results)

        if not os.path.exists(pred_path):
            # Write with header if file does not exist
            chunk_df.to_csv(pred_path, mode='w', index=False, header=True)
        else:
            # Append without header
            chunk_df.to_csv(pred_path, mode='a', index=False, header=False)

        # Compute metrics
        accuracy = accuracy_score(y_true, y_pred)
        f1 = f1_score(y_true, y_pred, average='macro', zero_division=0)
        metrics_entry = pd.DataFrame([{"step": idx + 1, "accuracy": accuracy, "f1_score": f1}])

        if not os.path.exists(metrics_path):
            metrics_entry.to_csv(metrics_path, mode='w', index=False, header=True)
        else:
            metrics_entry.to_csv(metrics_path, mode='a', index=False, header=False)

        # Reset for next chunk
        results, y_true, y_pred = [], [], []

        print(f"Checkpoint saved at index {idx + 1}")

# Save predictions
results_df = pd.DataFrame(results)
results_df.to_csv("bakllava_vqa_predictions.csv", index=False)

# Compute accuracy and F1 score
correct = 0
total = len(predictions)
predictions_lower = [str(pred).lower() for pred in predictions]
ground_truths_lower = [str(truth).lower() for truth in ground_truths]
for pred, truth in zip(predictions_lower, ground_truths_lower):
    if pred == truth:
        correct += 1
accuracy = (correct / total) * 100 if total > 0 else 0
# precision, recall, f1, _ = precision_recall_fscore_support(
#     ground_truths_lower, predictions_lower, average='macro', zero_division=0
# )

# Print results
print(f"\nTotal Questions: {total}")
print(f"Correct Predictions: {correct}")
print(f"Accuracy: {accuracy:.2f}%")
print(f"F1-Score: {((2*accuracy)/(1+accuracy)):.2f}%")
# print(f"Macro Precision: {precision:.2f}")
# print(f"Macro Recall: {recall:.2f}")
# print(f"Macro F1 Score: {f1:.2f}")
print(f"Skipped Entries: {len(skipped_entries)}")
if skipped_entries:
    print("\nSkipped Entries (due to timeout):")
    skipped_df = pd.DataFrame(skipped_entries)
    print(skipped_df)

# Display a few examples
results_df = pd.DataFrame({
    "Image Path": df["image_path"],
    "Question": df["question"],
    "Ground Truth": ground_truths,
    "Predicted": predictions
})
print("\nSample Predictions:")
print(results_df.head(10))

Found existing installation: rmm-cu12 25.2.0
Uninstalling rmm-cu12-25.2.0:
  Successfully uninstalled rmm-cu12-25.2.0
  Preparing metadata (setup.py) ... done
  Created wheel for timeout-decorator: filename=timeout_decorator-0.5.0-py3-none-any.whl size=5006 sha256=69750f97a3f7ba5f6a342343fe3fe3ba07eca06bf9dca60fbf6cc1e39268713c
  Stored in directory: /root/.cache/pip/wheels/aa/cd/d1/51736c6b95846b2613a520ce146a8f305c4016a987bc9faec7
Successfully built timeout-decorator


2025-05-18 01:34:59.957697: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747532100.186509      59 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747532100.251009      59 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


processor_config.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/589 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/505 [00:00<?, ?B/s]

...

Processing:   1%|          | 100/11972 [01:38<3:19:38,  1.01s/it]

[99] Truth: rectangular | Predicted: [/inst] | Match: ❌


Processing:   1%|          | 101/11972 [01:39<3:16:16,  1.01it/s]

[100] Truth: green | Predicted: [/inst] | Match: ❌


Processing:   1%|          | 112/11972 [01:50<3:14:50,  1.01it/s]

In [ ]:
print(len(predictions_lower))
print(len(ground_truths_lower))